<a href="https://colab.research.google.com/github/ZhangYHe/MyColabDL_Repo/blob/main/work2_loss_net.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##损失函数
损失函数选择为交叉熵损失函数。   
参数y_hat为预测的样本在10个类别上的预测概率,y为每个样本的真实类别数字。通过y索引得到预测样本在真实类别上的预测概率，取负对数，即为损失值。

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
def CE_loss(y_hat, y):
    return - torch.log(y_hat[range(len(y_hat)), y])


##Res-net 50网络(1)

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


class Residual(nn.Module):  
    def __init__(self, input_channels, num_channels_1, num_channels_2,
                 use_1x1conv=False, strides=1):
        #print(input_channels, num_channels_1, num_channels_2)
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels_1,
                               kernel_size=1, padding=0, stride=strides)
        self.conv2 = nn.Conv2d(num_channels_1, num_channels_1,
                               kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(num_channels_1, num_channels_2,
                               kernel_size=1, padding=0)
        # self.conv4 = nn.Conv2d(input_channels, num_channels_2,
        #                        kernel_size=1, padding=0, stride=1)
        if use_1x1conv:
            self.conv4 = nn.Conv2d(input_channels, num_channels_2,
                                   kernel_size=1, padding=0, stride=strides)
        else:
            self.conv4 = None
        self.bn1 = nn.BatchNorm2d(num_channels_1)
        self.bn2 = nn.BatchNorm2d(num_channels_1)
        self.bn3 = nn.BatchNorm2d(num_channels_2)
        self.bn4 = nn.BatchNorm2d(num_channels_2)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        # print('X1',X.shape)
        # Y=self.conv1(X)
        # Y=self.bn1(Y)
        # Y=F.relu(Y)
        Y = F.relu(self.bn2(self.conv2(X)))
        Y = self.bn3(self.conv3(Y))
        if self.conv4:
            X = self.conv4(X)
        #X =self.conv4(X)
        print('X',X.shape)
        print('Y',Y.shape)
        Y += X
        return F.relu(Y)


def resnet_block(input_channels, num_channels, num_residuals,first_block=False):
    blk = []
    for i in range(num_residuals):
        # if i == 0 and not first_block:
        #     blk.append(Residual(input_channels, num_channels,num_channels*4,
        #                         use_1x1conv=True, strides=2))
        #     print("blk",i,input_channels, num_channels,num_channels*4)
        # else:
        #     blk.append(Residual(input_channels, num_channels,num_channels*4))
        #     print("blk",i,input_channels, num_channels,num_channels*4)
        if i==0 and not first_block:
          blk.append(Residual(input_channels, num_channels,num_channels*4,
                                use_1x1conv=True, strides=2))
          #print("blk",i,input_channels, num_channels,num_channels*4)
        else:
          blk.append(Residual(num_channels*4, num_channels,num_channels*4))
          #print("blk",i,input_channels, num_channels,num_channels*4)
    return blk

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(*resnet_block(64, 64, 3, first_block=True))
b3 = nn.Sequential(*resnet_block(256, 128, 4))
b4 = nn.Sequential(*resnet_block(512, 256, 6))
b5 = nn.Sequential(*resnet_block(1024, 512, 3))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(2048, 10))

#检验输出
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
X1 torch.Size([1, 64, 56, 56])


RuntimeError: ignored

##Res-net 34网络

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


class Residual(nn.Module):  
    def __init__(self, input_channels, num_channels,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels,
                               kernel_size=3, padding=1, stride=strides)
        self.conv2 = nn.Conv2d(num_channels, num_channels,
                               kernel_size=3, padding=1)
        if use_1x1conv:
            self.conv3 = nn.Conv2d(input_channels, num_channels,
                                   kernel_size=1, stride=strides)
        else:
            self.conv3 = None
        self.bn1 = nn.BatchNorm2d(num_channels)
        self.bn2 = nn.BatchNorm2d(num_channels)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        if self.conv3:
            X = self.conv3(X)
        Y += X
        return F.relu(Y)

def resnet_block(input_channels, num_channels, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels,
                                use_1x1conv=True, strides=2))
        else:
            blk.append(Residual(num_channels, num_channels))
    return blk

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(*resnet_block(64, 64, 3, first_block=True))
b3 = nn.Sequential(*resnet_block(64, 128, 4))
b4 = nn.Sequential(*resnet_block(128, 256, 6))
b5 = nn.Sequential(*resnet_block(256, 512, 3))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(512, 10))

#检验输出
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 128, 28, 28])
Sequential output shape:	 torch.Size([1, 256, 14, 14])
Sequential output shape:	 torch.Size([1, 512, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 512, 1, 1])
Flatten output shape:	 torch.Size([1, 512])
Linear output shape:	 torch.Size([1, 10])


##Res-net 50 (2)

In [ ]:
import torch
from torch import nn
from torch.nn import functional as F


class Residual(nn.Module): 
    def __init__(self, input_channels, num_channels_1,num_channels_2,
                 use_1x1conv=False, strides=1):
        super().__init__()
        self.conv1 = nn.Conv2d(input_channels, num_channels_1,  kernel_size=1, padding=0, stride=strides)
        self.conv2 = nn.Conv2d(num_channels_1, num_channels_1,
                               kernel_size=3, padding=1, stride=1)
        self.conv3 = nn.Conv2d(num_channels_1, num_channels_2,
                               kernel_size=1, padding=0, stride=1)
        if use_1x1conv:
            self.conv4 = nn.Conv2d(input_channels, num_channels_2,
                                   kernel_size=1, stride=strides)
        else:
            self.conv4 = None
        self.bn1 = nn.BatchNorm2d(num_channels_1)
        self.bn2 = nn.BatchNorm2d(num_channels_1)
        self.bn3 = nn.BatchNorm2d(num_channels_2)

    def forward(self, X):
        Y = F.relu(self.bn1(self.conv1(X)))
        Y = self.bn2(self.conv2(Y))
        Y = self.bn3(self.conv3(Y))
        if self.conv4:
            X = self.conv4(X)
        Y += X
        return F.relu(Y)

def resnet_block(input_channels, num_channels_1,  num_channels_2, num_residuals,
                 first_block=False):
    blk = []
    for i in range(num_residuals):
        if i == 0 and not first_block:
            blk.append(Residual(input_channels, num_channels_1, num_channels_2,
                                use_1x1conv=True, strides=2))
        elif i == 0 and first_block:
            blk.append(Residual(input_channels, num_channels_1, num_channels_2,
                                use_1x1conv=True))
        else:
            blk.append(Residual(num_channels_2, num_channels_1, num_channels_2))
    return blk

b1 = nn.Sequential(nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3),
                   nn.BatchNorm2d(64), nn.ReLU(),
                   nn.MaxPool2d(kernel_size=3, stride=2, padding=1))
b2 = nn.Sequential(*resnet_block(64,  64, 256, 3, first_block=True))
b3 = nn.Sequential(*resnet_block(256, 128, 512, 4))
b4 = nn.Sequential(*resnet_block(512, 256, 1024, 6))
b5 = nn.Sequential(*resnet_block(1024,  512, 2048, 3))

net = nn.Sequential(b1, b2, b3, b4, b5,
                    nn.AdaptiveAvgPool2d((1,1)),
                    nn.Flatten(), nn.Linear(2048, 10))

#检验输出
X = torch.rand(size=(1, 1, 224, 224))
for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Sequential output shape:	 torch.Size([1, 64, 56, 56])
Sequential output shape:	 torch.Size([1, 256, 56, 56])
Sequential output shape:	 torch.Size([1, 512, 28, 28])
Sequential output shape:	 torch.Size([1, 1024, 14, 14])
Sequential output shape:	 torch.Size([1, 2048, 7, 7])
AdaptiveAvgPool2d output shape:	 torch.Size([1, 2048, 1, 1])
Flatten output shape:	 torch.Size([1, 2048])
Linear output shape:	 torch.Size([1, 10])


##Res-net 50 (3)

In [ ]:
class ConvBlock(nn.Module):
    def __init__(self, in_channel, f, filters, s):
        super(ConvBlock,self).__init__()
        F1, F2, F3 = filters
        self.stage = nn.Sequential(
            nn.Conv2d(in_channel,F1,1,stride=s, padding=0, bias=False),
            nn.BatchNorm2d(F1),
            nn.ReLU(True),
            nn.Conv2d(F1,F2,f,stride=1, padding=True, bias=False),
            nn.BatchNorm2d(F2),
            nn.ReLU(True),
            nn.Conv2d(F2,F3,1,stride=1, padding=0, bias=False),
            nn.BatchNorm2d(F3),
        )
        self.shortcut_1 = nn.Conv2d(in_channel, F3, 1, stride=s, padding=0, bias=False)
        self.batch_1 = nn.BatchNorm2d(F3)
        self.relu_1 = nn.ReLU(True)
        
    def forward(self, X):
        X_shortcut = self.shortcut_1(X)
        X_shortcut = self.batch_1(X_shortcut)
        X = self.stage(X)
        X = X + X_shortcut
        X = self.relu_1(X)
        return X    
    
class IndentityBlock(nn.Module):
    def __init__(self, in_channel, f, filters):
        super(IndentityBlock,self).__init__()
        F1, F2, F3 = filters
        self.stage = nn.Sequential(
            nn.Conv2d(in_channel,F1,1,stride=1, padding=0, bias=False),
            nn.BatchNorm2d(F1),
            nn.ReLU(True),
            nn.Conv2d(F1,F2,f,stride=1, padding=True, bias=False),
            nn.BatchNorm2d(F2),
            nn.ReLU(True),
            nn.Conv2d(F2,F3,1,stride=1, padding=0, bias=False),
            nn.BatchNorm2d(F3),
        )
        self.relu_1 = nn.ReLU(True)
        
    def forward(self, X):
        X_shortcut = X
        X = self.stage(X)
        X = X + X_shortcut
        X = self.relu_1(X)
        return X
    
class ResModel(nn.Module):
    def __init__(self, n_class):
        super(ResModel,self).__init__()
        self.stage1 = nn.Sequential(
            nn.Conv2d(3,64,7,stride=2, padding=3, bias=False),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.MaxPool2d(3,2,padding=1),
        )
        self.stage2 = nn.Sequential(
            ConvBlock(64, f=3, filters=[64, 64, 256], s=1),
            IndentityBlock(256, 3, [64, 64, 256]),
            IndentityBlock(256, 3, [64, 64, 256]),
        )
        self.stage3 = nn.Sequential(
            ConvBlock(256, f=3, filters=[128, 128, 512], s=2),
            IndentityBlock(512, 3, [128, 128, 512]),
            IndentityBlock(512, 3, [128, 128, 512]),
            IndentityBlock(512, 3, [128, 128, 512]),
        )
        self.stage4 = nn.Sequential(
            ConvBlock(512, f=3, filters=[256, 256, 1024], s=2),
            IndentityBlock(1024, 3, [256, 256, 1024]),
            IndentityBlock(1024, 3, [256, 256, 1024]),
            IndentityBlock(1024, 3, [256, 256, 1024]),
            IndentityBlock(1024, 3, [256, 256, 1024]),
            IndentityBlock(1024, 3, [256, 256, 1024]),
        )
        self.stage5 = nn.Sequential(
            ConvBlock(1024, f=3, filters=[512, 512, 2048], s=2),
            IndentityBlock(2048, 3, [512, 512, 2048]),
            IndentityBlock(2048, 3, [512, 512, 2048]),
        )
        self.pool = nn.AvgPool2d(2,2,padding=1)
        self.fc = nn.Sequential(
            nn.Linear(8192,n_class)
        )
    
    def forward(self, X):
        out = self.stage1(X)
        out = self.stage2(out)
        out = self.stage3(out)
        out = self.stage4(out)
        out = self.stage5(out)
        out = self.pool(out)
        out = out.view(out.size(0),8192)
        out = self.fc(out)
        return out